
# Regression:       Context $\Rightarrow$ Outcome

In [2]:
#imports
import numpy as np
import torch as th
# from torchsummary import summary
from tqdm.notebook import tqdm
import pandas as pd
import time

## Dataset

In [3]:
###################################################################################
###########                    MODIFY STUFF HERE                    ###############
###################################################################################

DATASETNAME = './data/10mln_data_samples_NEW.npz'

Keep in mind that in this dataset, the agent was assigned an object with some context

So, you cannot reuse these models on other agents with potentially different items' contexts

In [ ]:
# getting npz data
data = np.load(DATASETNAME, allow_pickle=True)
files = data.files
for f in files:
    print(f)
    # print(data[f].shape)
    # # if at least two dimensions, average on first
    # if data[f].ndim > 1:
    #     print(data[f][0].mean(axis=0))
    # else:
    #     print(data[f][0].mean())

contexts, values, bids, prices, outcomes, estimated_CTRs, won_masks = (data[f] for f in files)


In [ ]:
#### INSPECT DATASET ####

# AVERAGE VALUE OF OUTCOMES
# print(contexts.transpose(1,0).shape)
contexts1, contexts2, contexts3, contexts4, contexts5, contexts6 = contexts.transpose(1,0)
# df= pd.DataFrame.from_dict({item: data[item] for item in data.files}, orient='index')
# df.head()

d = {item: data[item] for item in data.files if data[item].ndim == 1}

d['contexts1'] = contexts1
d['contexts2'] = contexts2
d['contexts3'] = contexts3
d['contexts4'] = contexts4
d['contexts5'] = contexts5
d['contexts6'] = contexts6

df = pd.DataFrame.from_dict(d, orient='index').transpose()
df.head()

,values,bids,prices,outcomes,estimated_CTRs,won_mask,contexts1,contexts2,contexts3,contexts4,contexts5,contexts6
0,1.293715,0.036494,0.036494,0,0.028208,False,1.647339,0.917488,1.066935,0.047673,0.916655,1.0
1,1.293715,0.057794,0.057794,0,0.044673,False,-0.152193,-1.473888,1.028854,-1.93496,-0.239937,1.0
2,1.293715,0.031547,0.031547,0,0.024385,True,0.613123,-0.20033,-0.436868,0.519842,-0.476579,1.0
3,1.293715,0.016949,0.016949,0,0.013101,False,-0.474333,-1.944265,-1.307753,1.086831,-0.050604,1.0
4,1.293715,0.028522,0.028522,0,0.022046,False,-1.282649,-0.585658,-0.472588,0.586337,-0.663535,1.0


In [ ]:
# check data has some positive outcomes
print(df['prices'].sum())
print(df['outcomes'].sum())

38602.62506158464
14762


In [ ]:
# FUNCTIONS
def build_x_y(X, Y, train_test_ratio, train_validation_ratio):
    #TEST
    first_test_index = int(train_test_ratio * len(contexts))

    x = th.from_numpy(contexts.astype(np.float32))
    x_train = x[:first_test_index, :]
    x_test = x[first_test_index:, :]

    y = th.from_numpy(outcomes.astype(np.float32))
    y = y.reshape(-1,1)
    y_train = y[:first_test_index, :]
    y_test = y[first_test_index:, :]

    # VALIDATION
    first_val_index = int(train_validation_ratio * len(x_train))

    x_val = x_train[first_val_index:, :]
    x_train = x_train[:first_val_index, :]
    y_val = y_train[first_val_index:, :]
    y_train = y_train[:first_val_index, :]

    return x_train, y_train, x_val, y_val, x_test, y_test


def my_train(model, x_train, y_train, x_val, y_val, epochs, batch, device, loss, optimizer):

    running_train_loss = 0.
    last_train_loss = 0.
    running_val_loss = 0.
    last_val_loss = 0.

    model.train()
    model.to(device)
    x_train.to(device)
    y_train.to(device)

    # for i in tqdm(range(len(x_train))):
    for i in tqdm(range(epochs)):
        optimizer.zero_grad()
        y_pred = model(x_train)
        l = loss(y_pred, y_train)
        l.backward()
        optimizer.step()
        running_train_loss += l.item()
        l_val = loss(model(x_val), y_val)
        running_val_loss += l_val.item()

        if i % batch == 0:
            last_train_loss = running_train_loss / batch # loss per batch
            last_val_loss = running_val_loss / batch # loss per batch
            print('  batch {}\ttraining loss: {}'.format(i, last_train_loss), end=' ')
            print('\tvalidation loss: {}'.format(last_val_loss))
            running_train_loss = 0.
            running_val_loss = 0.

    return last_train_loss, last_val_loss


def my_test(model, x_test, y_test, loss):
    cumulative_loss = 0.

    for i in tqdm(range(len(x_test))):
        a = x_test[i]
        y_pred = model(a)
        #loss
        l = loss(y_pred, y_test[i])
        cumulative_loss += l.item()

    average_loss = cumulative_loss / len(x_test)

    return average_loss

## NN

In [ ]:
th.random.manual_seed(42)

# model parameters
layers = [6,16,4,1]
dropout = 0.2
activation = th.nn.ReLU()

# dataset amount
use_only = 100000

model_layers = []
for i in range(len(layers)-1):
    model_layers.append(th.nn.Linear(layers[i], layers[i+1]))
    if i < len(layers)-2:
        model_layers.append(th.nn.Dropout(dropout))
        model_layers.append(activation)
    else:
        model_layers.append(th.nn.Sigmoid())

model = th.nn.Sequential(*model_layers)

print(contexts.shape)

contexts_old = contexts
outcomes_old = outcomes
contexts = contexts[ :use_only ,  :]
outcomes = outcomes[ :use_only]

print(contexts.shape)

x_train, y_train, x_val, y_val, x_test, y_test = build_x_y(contexts, outcomes, 0.9, 0.9)

print("train:\t", x_train.shape, y_train.shape)
print("val:\t", x_val.shape, y_val.shape)
print("test:\t", x_test.shape, y_test.shape)


'''=======================
        TRAINING
======================='''
loss = th.nn.MSELoss()
optimizer = th.optim.Adam(model.parameters(), lr=0.001)
epochs = 10000
device = 'cpu'
# batch = int(epochs / 10)
batch = 100

last_train_loss, last_val_loss = my_train(model, x_train, y_train, x_val, y_val, epochs, batch, device, loss, optimizer)


'''=======================
        TESTING
======================='''
average_loss = my_test(model, x_test, y_test, loss)

print("average loss (error prob.) is: ", average_loss)


(1000000, 6)
(100000, 6)
train:	 torch.Size([81000, 6]) torch.Size([81000, 1])
val:	 torch.Size([9000, 6]) torch.Size([9000, 1])
test:	 torch.Size([10000, 6]) torch.Size([10000, 1])


  0%|          | 0/10000 [00:00<?, ?it/s]

  batch 0	training loss: 0.0033955705165863038 	validation loss: 0.003384522497653961
  batch 100	training loss: 0.29280158966779707 	validation loss: 0.2915407080948353
  batch 200	training loss: 0.16679205060005187 	validation loss: 0.16506456360220909
  batch 300	training loss: 0.08557244464755058 	validation loss: 0.08514108024537563
  batch 400	training loss: 0.06821904003620148 	validation loss: 0.06813966013491153
  batch 500	training loss: 0.06334397722035646 	validation loss: 0.06349836077541113
  batch 600	training loss: 0.060760054551064965 	validation loss: 0.06069764260202646
  batch 700	training loss: 0.05810908354818821 	validation loss: 0.057931699678301814
  batch 800	training loss: 0.03461921637877822 	validation loss: 0.034411395881325
  batch 900	training loss: 0.024314447287470103 	validation loss: 0.02426236368715763
  batch 1000	training loss: 0.023668208718299867 	validation loss: 0.023582434207201006
  batch 1100	training loss: 0.023423574548214674 	validation 

  0%|          | 0/10000 [00:00<?, ?it/s]

average loss (error prob.) is:  0.01302087077293522


In [ ]:
# if you want to save the model
ts = time.strftime("%Y%m%d-%H%M", time.localtime())
th.save(model, "./models/nn_ctr_estimator_test/nn_ctr_"+ts+".pt")

filename = "./models/nn_ctr_estimator_test/nn_ctr_"+ts+"_description.txt"
#open file
f = open(filename, "w")
#write
f.write("model description:\n")
f.write(str(model))
f.write("\n\n\n")
f.write("model parameters:\n")
f.write("layers: "+str(layers)+"\n")
f.write("dropout: "+str(dropout)+"\n")
f.write("activation: "+str(activation)+"\n")
f.write("\n\n")
f.write("training parameters:\n")
f.write("epochs: "+str(epochs)+"\n")
#f.write("batch: "+str(batch)+"\n") only used to display loss
f.write("optimizer: "+str(optimizer)+"\n")
f.write("loss: "+str(loss)+"\n")
f.write("\n\n")
f.write("results:\n")
f.write("last train loss: "+str(last_train_loss)+"\n")
f.write("last val loss: "+str(last_val_loss)+"\n")
f.write("average loss (error prob.) is: "+str(average_loss)+"\n")
#close file
f.close()

#### load model and fine-tune

In [ ]:
# MODEL NAME
MODELNAME = "./models/nn_ctr_estimator_test/nn_ctr_20230623-1641.pt"

loaded_model = th.load(MODELNAME)
loaded_model.eval()

already_trained = 100000
new_data_amt = 100000

new_x = contexts[already_trained : already_trained + new_data_amt, :]
new_y = outcomes[already_trained : already_trained + new_data_amt]

x_train, y_train, x_val, y_val, x_test, y_test = build_x_y(new_x, new_y, 0.9, 0.9)

loss = th.nn.MSELoss()
optimizer = th.optim.Adam(model.parameters(), lr=0.001)
epochs = 2000
device = 'cpu'
# batch = int(epochs / 10)
batch = 100

last_train_loss, last_val_loss = my_train(loaded_model, x_train, y_train, x_val, y_val, epochs, batch, device, loss, optimizer)

average_loss = my_test(loaded_model, x_test, y_test, loss)

print("average loss (error prob.) is: ", average_loss)

  0%|          | 0/2000 [00:00<?, ?it/s]

  batch 0	training loss: 0.0030190441757440567 	validation loss: 0.002998082898557186
  batch 5	training loss: 0.015104143507778644 	validation loss: 0.015088490210473537
  batch 10	training loss: 0.01511098798364401 	validation loss: 0.015060930512845516
  batch 15	training loss: 0.015102998912334442 	validation loss: 0.015083630755543708
  batch 20	training loss: 0.015099073573946954 	validation loss: 0.015075460076332092
  batch 25	training loss: 0.015111139602959156 	validation loss: 0.015076599828898907
  batch 30	training loss: 0.01510429810732603 	validation loss: 0.015065214782953262
  batch 35	training loss: 0.015100200287997722 	validation loss: 0.015094640478491784
  batch 40	training loss: 0.0150959812104702 	validation loss: 0.015072726272046566
  batch 45	training loss: 0.015100816637277603 	validation loss: 0.015073267742991447
  batch 50	training loss: 0.015106389857828617 	validation loss: 0.01507485844194889
  batch 55	training loss: 0.015101350843906403 	validation l

  0%|          | 0/10000 [00:00<?, ?it/s]

average loss (error prob.) is:  0.013035350477946522


## Gaussian Process Regressor

In [ ]:
from ModelsMine import BIGPR

